In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import math
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
%matplotlib inline
import re

import gensim

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.cluster import KMeans

In [2]:
df = pd.read_pickle('non_concat_review_combined')

In [4]:
df.drop_duplicates(inplace = True)

In [5]:
df.shape

(133459, 6)

In [6]:
df.to_pickle('single_review_no_duplicate')

In [8]:
df.reset_index(inplace = True)

In [11]:
df.drop('index', axis = 1, inplace = True)

In [12]:
df.head()

,review_id,business_id,stars,date,text,catergory
0,6I9xY8BIUYYELnVKepLduQ,--DaPTJW3-tB1vP-PfdTEg,3,2017-11-03,Does a reasonable job in terms of predictable ...,"Restaurants, Breakfast & Brunch"
1,llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,2013-09-14,Had breakfast here for the first time today. N...,"Restaurants, Breakfast & Brunch"
2,I5K5G7k_bjSUjWyx4gq3_Q,--DaPTJW3-tB1vP-PfdTEg,4,2012-06-04,A super pleasant place for a delicious breakfa...,"Restaurants, Breakfast & Brunch"
3,uyV7ozTQgJeiOztegK-VmQ,--DaPTJW3-tB1vP-PfdTEg,4,2013-07-06,"Decent breakfasty food. Decent eggs, toast and...","Restaurants, Breakfast & Brunch"
4,7gyqtrUA884_h6Fu_uAUOQ,--DaPTJW3-tB1vP-PfdTEg,4,2015-07-22,Great food. Great price. Was suspicious of r...,"Restaurants, Breakfast & Brunch"


In [13]:
review_text = list(df.text)

In [16]:
def ProcessTextforSentiment(text):
    text_1 = [re.sub("[^a-zA-Z]"," ", review) for review in text]
    text_2 = [text.lower().split() for text in text_1]
    return text_2

In [17]:
review_text_split_lower = ProcessTextforSentiment(review_text)

In [ ]:
bigram = gensim.models.Phrases(texts_out, min_count=10) # higher threshold fewer phrases.

trigram = gensim.models.Phrases(bigram[texts_out], min_count=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_model = gensim.models.phrases.Phraser(bigram)
trigram_model = gensim.models.phrases.Phraser(trigram)

In [23]:
review_text_1 = [trigram_model[bigram_model[t]] for t in review_text_split_lower]

In [25]:
#after process the text, use it to create the word2vec model
# Creating the model and setting values for the various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
#print("Training model....")
model_word2vec = word2vec.Word2Vec(review_text_1,
                          workers=num_workers,
                          size=num_features,
                          min_count=min_word_count,
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model_word2vec.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context_no_stopword_111818"
model_word2vec.save(model_name)

In [26]:
#transform each reveiw into a 300 D vector
index2word_set =  set(model_word2vec.wv.index2word)

# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features, index2word_set):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0

    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


In [28]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features,index2word_set):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
#         if counter%1000 == 0:
#             print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features, index2word_set)
        counter = counter+1
        
    return reviewFeatureVecs


In [29]:
review_vector = getAvgFeatureVecs(review_text_1, model_word2vec, num_features, index2word_set)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [30]:
review_vector.shape

(133459, 300)

In [31]:
#have to fillin na for next step training
def ProcessWordVec(array):
    df = pd.DataFrame(array)
    column_mean= df.mean()
    df.fillna(column_mean, inplace = True)
    return df


In [32]:
X = ProcessWordVec(review_vector)

In [35]:
#our response variable is the star from our dataframe
y = df.stars

In [37]:
#make sure X, y are matching
print (X.shape)
print (y.shape)

(133459, 300)
(133459,)


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [42]:
#from previous modeling use C =10
model_log = LogisticRegression(C = 10)
model_log.fit(X_train,y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [43]:
y_predict_log = model_log.predict(X_test)

In [44]:
MSE = np.mean(np.square(y_predict_log - y_test))

In [45]:
MSE

0.7639071795104673

In [ ]:
#after train the model, import split sentence dataframe to predict score and cluster 

In [115]:
df_split_sentence = pd.read_pickle('split_sentence_review_111318_no_duplicate.pkl')

In [116]:
df_split_sentence.shape

(1022705, 5)

In [117]:
df_split_sentence.head()

,text,review_id,business_id,stars,catergory
0,Does a reasonable job in terms of predictable ...,6I9xY8BIUYYELnVKepLduQ,--DaPTJW3-tB1vP-PfdTEg,3,"Restaurants, Breakfast & Brunch"
1,Significantly better than Boom which is down ...,6I9xY8BIUYYELnVKepLduQ,--DaPTJW3-tB1vP-PfdTEg,3,"Restaurants, Breakfast & Brunch"
2,Had breakfast here for the first time today,llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"
3,Nice little restaurant,llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"
4,"Prices are reasonable, food was fantastic! Ev...",llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"


In [50]:
text_sentence_review = list(df_split_sentence.text)

In [189]:
text_sentence_review_df = df_split_sentence.text

In [191]:
import pickle

In [192]:
with open('text_sentence_review_df.csv', 'wb') as f:
    pickle.dump(text_sentence_review_df, f)

In [52]:
def ProcessTextforSentiment_bitrigram(text):
    text_1 = [re.sub("[^a-zA-Z]"," ", review) for review in text]
    text_2 = [text.lower().split() for text in text_1]
    text_3 = [trigram_model[bigram_model[t]] for t in text_2]
    return text_3

In [53]:
text_sentence_review_sentiment = ProcessTextforSentiment_bitrigram(text_sentence_review)

In [118]:
text_sentence_review_sentiment[:3]

[['does',
  'a',
  'reasonable',
  'job',
  'in_terms',
  'of',
  'predictable',
  'diner',
  'food',
  'but',
  'not',
  'earth_shattering'],
 ['significantly_better_than',
  'boom',
  'which',
  'is',
  'down',
  'the',
  'road',
  'and',
  'also',
  'offers',
  'a',
  'similar',
  'selection'],
 ['had', 'breakfast', 'here', 'for', 'the', 'first_time', 'today']]

In [55]:
text_sentence_review_sentiment_vector = getAvgFeatureVecs(text_sentence_review_sentiment, model_word2vec, num_features, index2word_set)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [56]:
sentence_review_vector_df = ProcessWordVec(text_sentence_review_sentiment_vector)

In [57]:
#the vector with stop word is to predict rating
sentence_score = model_log.predict(sentence_review_vector_df)

In [119]:
sentence_score.shape


(1022705,)

In [58]:
text_no_stopword_sentence = pd.read_pickle('text_no_stopword_111818.pkl')

In [60]:
sentence_vector_no_stopword = getAvgFeatureVecs(text_no_stopword_sentence, model_word2vec, num_features, index2word_set)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [62]:
sentence_vector_no_stopword_df = ProcessWordVec(sentence_vector_no_stopword)

In [64]:
#The sentence_vector_no_stopword_df is used to cluster
km_300_10 = KMeans(n_clusters=10)
km_300_10.fit(sentence_vector_no_stopword_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [65]:
#similar_by_vector(vector, topn=10, restrict_vocab=None)
def Get_centroid_word(centroids,model):
    word_list = []
    for vector in centroids:
        words = model.similar_by_vector(vector, topn=10)
        word_list.append(words)
        
    return word_list
    


In [66]:
centroids_300_10 = km_300_10.cluster_centers_
topics_300_10 = Get_centroid_word(centroids_300_10,model_word2vec)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """
/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [67]:
topics_300_10

[[('service', 0.9496237635612488),
  ('customer_service', 0.7740272283554077),
  ('services', 0.7575777173042297),
  ('waitstaff', 0.7232248783111572),
  ('staff', 0.6943345069885254),
  ('environment', 0.5980492830276489),
  ('servers', 0.5728952884674072),
  ('food', 0.5634125471115112),
  ('atmosphere', 0.5516807436943054),
  ('ambience', 0.5482807159423828)],
 [('plaza', 0.7665568590164185),
  ('old_town_scottsdale', 0.7506887316703796),
  ('cafe', 0.7499853372573853),
  ('garage', 0.7492340803146362),
  ('mall', 0.7447710037231445),
  ('coffee_shop', 0.7407407760620117),
  ('situated', 0.7380146384239197),
  ('old_town', 0.7376625537872314),
  ('eatery', 0.7370427250862122),
  ('main_street', 0.7365387082099915)],
 [('iced_tea', 0.8001436591148376),
  ('ice_tea', 0.7913293838500977),
  ('vodka', 0.7864537239074707),
  ('latte', 0.7760728597640991),
  ('champagne', 0.7729272842407227),
  ('tapioca', 0.772567093372345),
  ('ice', 0.7704976797103882),
  ('carafe', 0.7704733610153198)

In [70]:
sentence_vector_no_stopword_df_array = sentence_vector_no_stopword_df.values

In [71]:
sentence_vector_no_stopword_df_array

array([[ 0.00292754, -0.04625789, -0.01106922, ...,  0.01326184,
         0.00922822, -0.01020616],
       [-0.04022259,  0.00194782, -0.01063114, ..., -0.04298754,
         0.07767633, -0.06974918],
       [ 0.08400548,  0.0061196 , -0.02723265, ...,  0.03101365,
        -0.03321704, -0.04189218],
       ...,
       [ 0.03000038, -0.03246419,  0.14260128, ...,  0.09557474,
         0.0028195 , -0.01137816],
       [-0.02158902,  0.00023091, -0.05066021, ...,  0.12079367,
        -0.01153452,  0.0833022 ],
       [-0.00718688, -0.05419859,  0.01343594, ...,  0.08287276,
        -0.03807875,  0.14380291]], dtype=float32)

In [74]:
cosine_similarity = []
for vector in sentence_vector_no_stopword_df_array:
    similarity = google_model.cosine_similarities(vector, centroids_300_10)
    cosine_similarity.append(similarity)
    

cosine_similarity_array = np.array(cosine_similarity)


In [76]:
cosine_similarity_array.shape

(1022705, 10)

In [77]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [78]:
softmax_cosine_similarity = [softmax(row) for row in cosine_similarity_array]
softmax_cosine_similarity_array = np.array(softmax_cosine_similarity)

In [79]:
softmax_cosine_similarity_array.shape

(1022705, 10)

In [80]:
def AssignTopics(topics):
    topic_assign = np.zeros((1022705,10))
    for index, topic in enumerate (topics):
        for index_1, item in enumerate(topic):
            if item > 0.12:
                topic_assign[index][index_1] =1
    return topic_assign
                

In [81]:
cluster_assign = AssignTopics(softmax_cosine_similarity_array)

In [82]:
cluster_assign[:5,:]

array([[0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 1., 0.]])

In [83]:
cluster_assign_df = pd.DataFrame(cluster_assign)

In [84]:
cluster_assign_df.columns = ['service', 'location', 'drink', 'sauce', 'meal', 'occation', 'meat', 'place', 'food', 'employee']

In [93]:
def topic_score_assign_combine(topic_assignment, score):
    
    df = pd.concat([topic_assignment,score], axis = 1)
    topic_score_df = pd.DataFrame()
    topic_score_df['drink_score'] = df['drink']*df['score']
    topic_score_df['meal_score'] = df['meal']*df['score']
    topic_score_df['weekend_score'] = df['occation']*df['score']
    topic_score_df['location_score'] = df['location']*df['score']
    topic_score_df['employee_score'] = df['employee']*df['score']
    topic_score_df['place_score'] = df['place']*df['score']
    topic_score_df['service_score'] = df['service']*df['score']
    topic_score_df['food_score'] = df['food']*df['score']
    topic_score_df['meat_score'] = df['meat']*df['score']
    topic_score_df['flavor_score'] = df['sauce']*df['score']
    topic_score_df.replace(0, np.nan, inplace = True)
    topic_score_combine_business = pd.concat([df_split_sentence, topic_score_df], axis =1)
    
    return topic_score_combine_business

In [107]:
df = pd.concat([cluster_assign_df,sentence_score_df], axis = 1)

In [109]:
df['drink_score'] = df['drink']*df['score']
df['meal_score'] = df['meal']*df['score']
df['occation_score'] = df['occation']*df['score']
df['location_score'] = df['location']*df['score']
df['employee_score'] = df['employee']*df['score']
df['place_score'] = df['place']*df['score']
df['service_score'] = df['service']*df['score']
df['food_score'] = df['food']*df['score']
df['meat_score'] = df['meat']*df['score']
df['flavor_score'] = df['sauce']*df['score']

In [111]:
df.columns

Index(['service', 'location', 'drink', 'sauce', 'meal', 'occation', 'meat',
       'place', 'food', 'employee', 'score', 'drink_score', 'meal_score',
       'occation_score', 'location_score', 'employee_score', 'place_score',
       'service_score', 'food_score', 'meat_score', 'flavor_score'],
      dtype='object')

In [112]:
df_1 = df[['drink_score', 'meal_score','occation_score', 'location_score', 'employee_score', 'place_score',
           'service_score', 'food_score', 'meat_score', 'flavor_score']]

In [129]:
df_split_sentence.reset_index(inplace= True)

In [130]:
#before concat two dataframe, remember to check the index, if the index did not align, it will mess up with the score
#assignment

df_2 = pd.concat([df_1,df_split_sentence], axis=1)

In [131]:
df_2.shape

(1022705, 16)

In [133]:
df_2.drop('index', axis =1, inplace = True)

In [134]:
df_2.head()

,drink_score,meal_score,occation_score,location_score,employee_score,place_score,service_score,food_score,meat_score,flavor_score,text,review_id,business_id,stars,catergory
0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,Does a reasonable job in terms of predictable ...,6I9xY8BIUYYELnVKepLduQ,--DaPTJW3-tB1vP-PfdTEg,3,"Restaurants, Breakfast & Brunch"
1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,Significantly better than Boom which is down ...,6I9xY8BIUYYELnVKepLduQ,--DaPTJW3-tB1vP-PfdTEg,3,"Restaurants, Breakfast & Brunch"
2,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Had breakfast here for the first time today,llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"
3,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Nice little restaurant,llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"
4,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,"Prices are reasonable, food was fantastic! Ev...",llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"


In [137]:
df_2.replace(0, np.nan, inplace = True)

In [148]:
sample = df_2[df_2.review_id == '-0kYPgtHInieMmF6bkdSDA']

In [149]:
for i in sample.text:
    print (i)
    print ('\n')

The lady that took our order sat on the phone for 10 minutes talking while our food got cold along with some near by patrons food as well


 Horrible service


 The only thing hot was the white rice




In [150]:
sample

,drink_score,meal_score,occation_score,location_score,employee_score,place_score,service_score,food_score,meat_score,flavor_score,text,review_id,business_id,stars,catergory
250522,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,The lady that took our order sat on the phone ...,-0kYPgtHInieMmF6bkdSDA,2xbxXWeu3tpZEJdGTTGbLg,1,"Chinese, Restaurants"
250523,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Horrible service,-0kYPgtHInieMmF6bkdSDA,2xbxXWeu3tpZEJdGTTGbLg,1,"Chinese, Restaurants"
250524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,The only thing hot was the white rice,-0kYPgtHInieMmF6bkdSDA,2xbxXWeu3tpZEJdGTTGbLg,1,"Chinese, Restaurants"


In [138]:

topic_score_groupby_business = df_2.groupby('business_id').mean()

In [144]:
df_2.to_pickle('111918_second_draft_word2vec_topicmodeling_sentiment')

In [100]:
business_sample

,text,review_id,business_id,stars,catergory,drink_score,meal_score,weekend_score,location_score,employee_score,place_score,service_score,food_score,meat_score,flavor_score
446348,It's one of those places you go to when you do...,JTnQfIUDtGqGIGIvLoMuUw,5NzUIglhCrq88ojer2Q2jw,4.0,"Chinese, Restaurants, Halal",NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446349,"Food is okay for the price, but tell them to ...",JTnQfIUDtGqGIGIvLoMuUw,5NzUIglhCrq88ojer2Q2jw,4.0,"Chinese, Restaurants, Halal",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
446350,It takes a lot for me to give someone a 1 star...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
446351,The food was not bad - pretty much like any o...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
446352,The service was HORRIBLE - quite one of the w...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446353,"The server had attitude from the get go, star...",DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
446354,"When she started to bring out the food, she w...",DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
446355,She never checked in on us to see if everythi...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
446356,We ended up going up to her to cancel one of ...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
446357,"She didn't bring us the check, we had to go a...",DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1.0,"Chinese, Restaurants, Halal",NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
#the sentiment analysis is still a bit off, in different version of analysis.
#Try to use the google model to see wether prediction will be better. 


In [155]:
index2word_set = set(google_model.wv.index2word)
#transform each reveiw into a 300 D vector


# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features, index2word_set):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0

    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [154]:
google_model

In [157]:
review_vector_google = getAvgFeatureVecs(review_text_1, google_model, num_features, index2word_set)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [158]:
review_vector_google_df = ProcessWordVec(review_vector_google)

In [159]:
review_vector_google_df.shape

(133459, 300)

In [160]:
X = review_vector_google_df

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [163]:
model_log_google = LogisticRegression(C=10)
model_log_google.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [165]:
y_predict_google = model_log_google.predict(X_test)

In [166]:
MSE_log_google = np.mean(np.square(y_predict_google - y_test))

In [167]:
MSE_log_google

0.955088324780891

In [168]:
text_sentence_review_sentiment_vector_google = getAvgFeatureVecs(text_sentence_review_sentiment, google_model, num_features, index2word_set)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [169]:
sentence_sentiment_google = ProcessWordVec(text_sentence_review_sentiment_vector_google)

In [170]:
sentence_sentiment_google.shape

(1022705, 300)

In [171]:
y_sentence_google_score = model_log_google.predict(sentence_sentiment_google)

In [173]:
y_sentence_google_score_df = pd.DataFrame(y_sentence_google_score)

In [175]:
y_sentence_google_score_df.columns = ['score']

In [178]:
df_2.head()

,drink_score,meal_score,occation_score,location_score,employee_score,place_score,service_score,food_score,meat_score,flavor_score,text,review_id,business_id,stars,catergory
0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,Does a reasonable job in terms of predictable ...,6I9xY8BIUYYELnVKepLduQ,--DaPTJW3-tB1vP-PfdTEg,3,"Restaurants, Breakfast & Brunch"
1,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Significantly better than Boom which is down ...,6I9xY8BIUYYELnVKepLduQ,--DaPTJW3-tB1vP-PfdTEg,3,"Restaurants, Breakfast & Brunch"
2,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Had breakfast here for the first time today,llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"
3,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nice little restaurant,llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"
4,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,"Prices are reasonable, food was fantastic! Ev...",llsD_56ZeA4sE5RleBJP5g,--DaPTJW3-tB1vP-PfdTEg,5,"Restaurants, Breakfast & Brunch"


In [181]:
google_df_2_combine = pd.concat([df_2, y_sentence_google_score_df], axis =1 )

In [182]:
google_df_2_combine.shape

(1022705, 16)

In [187]:
sample = google_df_2_combine[google_df_2_combine.review_id == 'DQbvLNM2JcBfBI2nS53WhA']

In [193]:
sample.head()

,drink_score,meal_score,occation_score,location_score,employee_score,place_score,service_score,food_score,meat_score,flavor_score,text,review_id,business_id,stars,catergory,score
302136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,It takes a lot for me to give someone a 1 star...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1,"Chinese, Restaurants, Halal",3
302137,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,The food was not bad - pretty much like any o...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1,"Chinese, Restaurants, Halal",3
302138,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,The service was HORRIBLE - quite one of the w...,DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1,"Chinese, Restaurants, Halal",1
302139,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,"The server had attitude from the get go, star...",DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1,"Chinese, Restaurants, Halal",1
302140,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,"When she started to bring out the food, she w...",DQbvLNM2JcBfBI2nS53WhA,5NzUIglhCrq88ojer2Q2jw,1,"Chinese, Restaurants, Halal",1


In [ ]:
#still need to work on sentiment analysis, maybe have my own sentence data

In [ ]:
business_sample_1.rename(columns = {'drink_score': 'drink', 'price_score': 'price', 'meal_score': 'meal', 'experience_score': 'experience',
                                       'menu_score': 'menu', 'location_score': 'location', 'food_score': 'food', 'service_score': 'service', 'line_score': 'line',
                                       'place_score': 'place'}, inplace = True)

In [ ]:
plt.figure(figsize = (15,6))
plt.bar(business_sample_1.columns, business_sample_1.loc['5NzUIglhCrq88ojer2Q2jw'], color = 'red')
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 20)

plt.savefig('resturant_sample.svg', format= 'svg')


In [ ]:
topic_score_groupby_business['food_serive_difference'] = abs(topic_score_groupby_business['food_score'] - topic_score_groupby_business['service_score'])

In [ ]:
topic_score_groupby_business.sort_values(by = 'food_serive_difference', ascending= False)